<a href="https://colab.research.google.com/github/Fathimath-Rifna-VK/fmml2021/blob/main/Module_10_FMML_M10_Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to `pytorch-lightning`

Machine learning is basically updating a float vector smartly. We don't actually need pytorch for that. There is a lot of ML stuff that runs on plain `numpy`. We use `pytorch` because it makes our lives easier.

As `pytorch` is a framework for machine learning (basically a library for managing tensors), `pytorch-lightning` is a wrapper for `pytorch` itself!

PyTorch Lightning is just organized PyTorch!

Lightning structures PyTorch code with these principles:

<div align="center">
  <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/philosophies.jpg" max-height="250px">
</div>

## Advantages over unstructured PyTorch

* Models become hardware agnostic
* Code is clear to read because engineering code is abstracted away
* Easier to reproduce
* Make fewer mistakes because lightning handles the tricky engineering
* Keeps all the flexibility (LightningModules are still PyTorch modules), but removes a ton of boilerplate
* Lightning has dozens of integrations with popular machine learning tools.

In [ ]:
!pip install pytorch-lightning

In [ ]:
import os

import pandas as pd
import pytorch_lightning as pl
from skimage import io
from sklearn.preprocessing import LabelEncoder
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

We'll be using the same example as the last lab, but now with `pytorch-lightning`

In [ ]:
# download the example dataset
!rm -r data cifar10.zip
!mkdir data
# slightly modified version of https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!wget https://web.iiit.ac.in/~yoogottam.khandelwal/cifar10.zip
!unzip cifar10.zip -d data | tail

In [ ]:
class CIFAR10Dataset(Dataset):
    def __init__(self, root_dir="./data", transforms=None):
        """
        Args:
            root_dir   (string)  : Directory with all the images and the csv file
            transforms (Callable): image goes through these transforms
        """
        # use pandas to read the csv
        # we are only using 1/10th of the full data because CIFAR10 is HUGE
        # otherwise, we wouldn't be able to train this
        self.data = pd.read_csv(f"{root_dir}/data.csv").sample(frac=0.1, random_state=42)
        # this is where the images are stored
        self.root_dir = root_dir

        # label encoder
        label_encoder = LabelEncoder()
        self.data["label_numeric"] = label_encoder.fit_transform(self.data["label"])

        # transformations to the dataset
        # we'll come to this soon
        if transforms is None:
            # an identity function, returns it's argument
            transforms = lambda x: x
        self.transforms = transforms

    def __len__(self):
        """
        This should return the number of items in the dataset
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        This should return the item at index `idx` from the dataset
        """
        data = self.data.iloc[idx]
        img_name = os.path.join(self.root_dir, data["image_path"])
        # read the image
        image = io.imread(img_name)
        label = data["label"]
        label_numeric = data["label_numeric"]

        # send the image through the provided transformations
        image = self.transforms(image)
        return image, label_numeric, label

Now, we'll use a `LightningModule` to define our model.

A `LightningModule` is basically a torch `nn.Module` but using this allows us to structure our code in a better way.

No change in the model code at all (except now we are inheriting from a different class)

On top of that, we configure the optimizers here

In [ ]:
class CIFAR10Module(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features=3072, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=10),
        )
        self.loss = nn.CrossEntropyLoss()
    
    def forward(self, x):
        return self.layers(x)
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)
    
    def training_step(self, batch, batch_idx):
        # pytorch-lightning will take care of
        # moving stuff to the correct device
        images, y, _ = batch
        x = images.view(images.size(0), -1)
        out = model(x)
        loss = self.loss(out, y)
        self.log("train_loss", loss)
        return loss

In [ ]:
cifar10_ds = CIFAR10Dataset(transforms=transforms.Compose([
    transforms.ToTensor(),
    # the order is flipped because RandomHorizontalFlip expects a tensor
    # almost always we'll convert the data to a tensor first
    transforms.RandomHorizontalFlip(),
]))

train_dl = DataLoader(cifar10_ds, batch_size=256, shuffle=True, num_workers=2)

# Training

Now that we have a `LightningModule`, we don't need to write the training code ourselves. `pytorch-lightning` provides us a `Trainer`

In [ ]:
trainer = pl.Trainer(max_epochs=30)
model = CIFAR10Module()

trainer.fit(model, train_dl)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

# References
 - https://github.com/PyTorchLightning/pytorch-lightning
 - https://towardsdatascience.com/from-pytorch-to-pytorch-lightning-a-gentle-introduction-b371b7caaf09
 - https://pytorch.org/vision/stable/transforms.html